#Introduction

1. load the data
2. data categorization
3. modeling

#1. load the data

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json

from datetime import timedelta
from tqdm.notebook import tqdm
from collection import Counter
from heapq import nlargest

#그림 그리기
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_path = Path("/kaggle/input/otto-recommender-system")

train= data_path/'train.jsonl'
test= data_path/'test.jsonl'

sample_sub = Path("/kaggle/input/otto-recommender-system/sample_submission.csv")

In [ ]:
with open(train,'r') as f :
  print(f'total lines in train data is : {len(f.readlines())}')
  

chunk session for train

In [ ]:
def read_jsonl(target: str) -> pd.DataFrame():
    sessions = pd.DataFrame()
    chunks = pd.read_json(data_path / f'{target}.jsonl', lines=True, chunksize=100_000)

    for e, chunk in enumerate(chunks):
        event_dict = {
            'session': [],
            'aid': [],
            'ts': [],
            'type': [],
        }
        if e < 2:
            for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
                for event in events:
                    event_dict['session'].append(session)
                    event_dict['aid'].append(event['aid'])
                    event_dict['ts'].append(event['ts'])
                    event_dict['type'].append(event['type'])
            chunk_session = pd.DataFrame(event_dict)
            sessions = pd.concat([sessions, chunk_session])
        else:
            break
    return sessions.reset_index(drop=True)

train_sessions = read_jsonl('train')
test_sessions = read_jsonl('test')

#2. categorization

In [ ]:
#extract info from each session and make it to the df

action_counts_list, article_id_list, session_length_ts_list, session_length_action_list = [[] for i in range(4)]

overall_action_counts = {}
overall_article_id_counts = {}

for i, row in tqdm(sample_df_train_df.iterrows(), total=len(sample_df_train_df)) :
  actions = row['events']

  action_counts = {}
  article_id_counts = {}

  action in actions :
  action_counts[action['type']] = action_counts.get(action['type'],0) +1
  article_id_counts[action['aid']] = article_id_counts.get(action['aid'],0) +1

  overall_action_counts[action['type']] = overall_action_counts.get(action['type'],0) +1
  overall_article_id_counts[action['aid']] = overall_article_id_counts.get(actio['aid'],0) +1

  session_length_time = action['ts'][-1] - action['ts'][0]

  #add to list
  action_counts_list.append(action_counts)
  article_id_list.append(article_id_counts)
  session_length_ts_list.append(session_length_time)
  session_length_action_list.append(len(actions))

sample_train_df['action_counts'] = action_counts_list
sample_train_df['article_id_list'] =article_id_list
sample_train_df['session_length_time'] =session_length_time_list
sample_train_df['session_length_hours'] = sample_train_df['session_length_time']*2.77778e-7
sample_train_df['session_length_action'] = session_length_action_list

#3. word2vec

In [ ]:
from nvtabular import *
from merlin.schema.tags import Tags
import polars as pl
import xgboost as xgv

from merlin.core.utils import Distributed
from merlin.models.xgb import XGBoost
from nvtabular.ops import AddTags